## Introduction to Quimb backend in QiboTN

#### Some imports

In [1]:
import time
import numpy as np
# from scipy import stats

# import qibo
from qibo import Circuit, gates, hamiltonians
from qibo.backends import construct_backend

#### Some hyper parameters

In [2]:
import cotengra as ctg
ctg_opt = ctg.ReusableHyperOptimizer(
    max_time=10,
    minimize='combo',
    slicing_opts=None,
    parallel=True,
    progbar=True
)


In [3]:
# construct qibotn backend
quimb_backend = construct_backend(backend="qibotn", platform="quimb")

# set number of qubits
nqubits = 4

# set numpy random seed
np.random.seed(42)

quimb_backend.setup_backend_specifics(qimb_backend="jax")

#### Constructing a parametric quantum circuit

In [4]:
def build_circuit(nqubits, nlayers):
    """Construct a parametric quantum circuit."""
    circ = Circuit(nqubits)
    for _ in range(nlayers):
        for q in range(nqubits):
            circ.add(gates.RY(q=q, theta=0.))
            circ.add(gates.RZ(q=q, theta=0.))
        [circ.add(gates.CNOT(q%nqubits, (q+1)%nqubits) for q in range(nqubits))]
    circ.add(gates.M(*range(nqubits)))
    return circ

In [5]:
circuit = build_circuit(nqubits=nqubits, nlayers=3)
circuit.draw()

0: ─RY─RZ─o─────X─RY─RZ─o─────X─RY─RZ─o─────X─M─
1: ─RY─RZ─X─o───|─RY─RZ─X─o───|─RY─RZ─X─o───|─M─
2: ─RY─RZ───X─o─|─RY─RZ───X─o─|─RY─RZ───X─o─|─M─
3: ─RY─RZ─────X─o─RY─RZ─────X─o─RY─RZ─────X─o─M─


In [6]:
# Setting random parameters
circuit.set_parameters(
    parameters=np.random.uniform(-np.pi, np.pi, len(circuit.get_parameters())),
)

#### Setting up the tensor network simulator

Depending on the simulator, various parameters can be set. One can customize the tensor network execution via the `backend.configure_tn_simulation` function, whose face depends on the specific backend provider.

In [7]:
# Customization of the tensor network simulation in the case of quimb backend
# Here we use only some of the possible arguments
quimb_backend.configure_tn_simulation(
    #ansatz="MPS",
    max_bond_dimension=10
)

#### Executing through the backend

The `backend.execute_circuit` method can be used then. We can simulate results in three ways:
1. reconstruction of the final state only if `return_array` is set to `True`;
2. computation of the relevant probabilities of the final state.
3. reconstruction of the relevant state's frequencies (only if `nshots` is not `None`).

In [8]:
# # Simple execution (defaults)
outcome = quimb_backend.execute_circuit(circuit=circuit, nshots=100, return_array=True)

# # Print outcome
vars(outcome)

/home/mattia/main_env/lib/python3.12/site-packages/quimb/tensor/circuit.py:215: SyntaxWarning: Unsupported operation ignored: creg
  warnings.warn(
/home/mattia/main_env/lib/python3.12/site-packages/quimb/tensor/circuit.py:215: SyntaxWarning: Unsupported operation ignored: measure
  warnings.warn(
/home/mattia/main_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'nqubits': 4,
 'backend': qibotn (quimb),
 'measures': Counter({'1010': 9,
          '0100': 8,
          '1101': 15,
          '1011': 4,
          '1111': 12,
          '1000': 13,
          '0000': 8,
          '0010': 6,
          '0011': 6,
          '0101': 8,
          '1110': 5,
          '0110': 5,
          '0111': 1}),
 'measured_probabilities': {'1101': np.float64(0.12331159869893256),
  '1000': np.float64(0.11330883548333587),
  '1111': np.float64(0.10184806171791962),
  '1010': np.float64(0.03872758515126756),
  '0100': np.float64(0.07142939529687138),
  '0000': np.float64(0.08390937969317269),
  '0101': np.float64(0.05622305772698622),
  '0010': np.float64(0.09466860481989385),
  '0011': np.float64(0.07571277233522114),
  '1110': np.float64(0.07174919872959985),
  '0110': np.float64(0.05146064807369214),
  '1011': np.float64(0.053499396925872744),
  '0111': np.float64(0.04029185074729259)},
 'prob_type': 'default',
 'statevector': Array([[ 0.08809624-0.27594998j],
     

---

One can access to the specific contents of the simulation outcome.

In [9]:
print(f"Probabilities:\n {outcome.probabilities()}\n")
print(f"State:\n {outcome.state()}\n")

Probabilities:
 {'1101': np.float64(0.12331159869893256), '1000': np.float64(0.11330883548333587), '1111': np.float64(0.10184806171791962), '1010': np.float64(0.03872758515126756), '0100': np.float64(0.07142939529687138), '0000': np.float64(0.08390937969317269), '0101': np.float64(0.05622305772698622), '0010': np.float64(0.09466860481989385), '0011': np.float64(0.07571277233522114), '1110': np.float64(0.07174919872959985), '0110': np.float64(0.05146064807369214), '1011': np.float64(0.053499396925872744), '0111': np.float64(0.04029185074729259)}

State:
 [[ 0.08809624-0.27594998j]
 [-0.05174781+0.04471217j]
 [ 0.00470147+0.30764672j]
 [-0.27208942+0.0409893j ]
 [ 0.18807822+0.18988408j]
 [ 0.2237706 +0.07842042j]
 [-0.18900308+0.12545314j]
 [ 0.17105256-0.10503749j]
 [ 0.24859734-0.22695419j]
 [-0.0411739 -0.06230037j]
 [ 0.17371392-0.09247189j]
 [-0.22748128+0.0418529j ]
 [ 0.09444095+0.06846087j]
 [-0.21784972-0.2754144j ]
 [-0.17359753+0.20399286j]
 [-0.01729754-0.31866732j]]



### Compute expectation values

Another important feature of this backend is the `expectation` function. In fact, we can compute expectation values of given observables thorugh a Qibo-friendly interface.

---

Let's start by importing some symbols, thanks to which we can build our observable.

In [10]:
import numpy as np
import jax
from qibo.backends import construct_backend
from qibo import Circuit, gates

In [11]:
# construct qibotn backend
quimb_backend = construct_backend(backend="qibotn", platform="quimb")

quimb_backend.setup_backend_specifics(
    qimb_backend    ="jax", 
    contractions_optimizer='auto-hq'
    )

quimb_backend.configure_tn_simulation(
    max_bond_dimension=10
)

In [19]:
# define Hamiltonian
operators = ["xzy", "yxzy", "zy"]
qubits = ["011", "0112", "01"]
coefficients = ["1", "2", "j"]
hamiltonian = (operators, qubits, coefficients)

In [18]:
# define circuit
def build_circuit(nqubits, nlayers):
    circ = Circuit(nqubits)
    for layer in range(nlayers):
        for q in range(nqubits):
            circ.add(gates.RY(q=q, theta=0.))
            circ.add(gates.RZ(q=q, theta=0.))
            circ.add(gates.RX(q=q, theta=0.))
        for q in range(nqubits - 1):
            circ.add(gates.CNOT(q, q + 1))
            circ.add(gates.SWAP(q, q + 1))
    circ.add(gates.M(*range(nqubits)))
    return circ

def build_circuit_problematic(nqubits, nlayers):
    circ = Circuit(nqubits)
    for _ in range(nlayers):
        for q in range(nqubits):
            circ.add(gates.RY(q=q, theta=0.))
            circ.add(gates.RZ(q=q, theta=0.))
        [circ.add(gates.CNOT(q%nqubits, (q+1)%nqubits) for q in range(nqubits))]
    circ.add(gates.M(*range(nqubits)))
    return circ


nqubits = 4
circuit = build_circuit(nqubits=nqubits, nlayers=3)


In [20]:
start = time.time()
expval = quimb_backend.expectation(
        circuit=circuit,
        operators_list=hamiltonian[0],
        sites_list=hamiltonian[1],
        coeffs_list=hamiltonian[2]
    )

elapsed = time.time() - start
print(f"Expectation value: {expval}")
print(f"Elapsed time: {elapsed:.4f} seconds")

Expectation value: 0.0
Elapsed time: 0.1071 seconds


Try with Qibo (which is by default using the Qibojit backend)


In [22]:
from qibo.symbols import Z, X, I
# We can create a symbolic Hamiltonian
form = 0.5 * Z(0) * Z(1) +- 1.5 *  X(0) * Z(2) + Z(3)
sym_hamiltonian = hamiltonians.SymbolicHamiltonian(form)

#  Let's show it
sym_hamiltonian.form

# Compute expectation value
start = time.time()
result = sym_hamiltonian.expectation(circuit().state())
elapsed = time.time() - start
print(f"Expectation value: {result}")
print(f"Elapsed time: {elapsed:.4f} seconds")

Expectation value: 1.5
Elapsed time: 0.0501 seconds


They match! 🥳

We can also compute gradient of expectation function

In [23]:
def f(circuit, hamiltonian, params):
    circuit.set_parameters(params)
    return quimb_backend.expectation(
        circuit=circuit,
        operators_list=hamiltonian[0],
        sites_list=hamiltonian[1],
        coeffs_list=hamiltonian[2]
    )

parameters = np.random.uniform(-np.pi, np.pi, size=len(circuit.get_parameters()))
print(jax.grad(f, argnums=2)(circuit, hamiltonian, parameters))


[ 8.19939339e-10 -3.14190913e-08 -2.99498648e-09 -1.03641796e-07
  8.48652704e-10  1.00297093e-07 -6.75429277e-08 -9.78565140e-09
 -5.11915417e-08  1.29225235e-08 -7.44280655e-08 -3.49115048e-08
 -4.98508879e-09  6.80729357e-08 -3.29755920e-08  4.20008526e-08
 -2.89742630e-08  1.18602941e-07 -2.88252178e-08  5.57985391e-09
 -3.17434115e-08 -1.03342952e-08  1.34079716e-08 -7.05437886e-09
 -4.34059650e-08 -2.18019203e-08 -5.36932561e-08 -6.38544009e-08
  5.85312279e-08  8.45709067e-08 -1.12777876e-09 -6.41545981e-08
  7.25317406e-08  4.10035668e-08 -1.29046382e-08  6.07501676e-08]
